In [2]:
from Dataset.Dataset import CSVLoader
from compoundFeaturization.rdkitFingerprints import MorganFingerprint
from featureSelection.baseFeatureSelector import LowVarianceFS
from splitters.splitters import SingletaskStratifiedSplitter
from models.sklearnModels import SklearnModel
from metrics.Metrics import Metric
from metrics.metricsFunctions import roc_auc_score, precision_score, accuracy_score, confusion_matrix, classification_report
from parameterOptimization.HyperparameterOpt import GridHyperparamOpt

from sklearn.ensemble import RandomForestClassifier

In [3]:
#Load Dataset
dataset = CSVLoader('preprocessed_dataset_wfoodb.csv', 'Smiles', ['Class'], 'ID')#, chunk_size=2000)
dataset.get_shape()

(23290,) (23290,) (0,) (23290,)


((23290,), (23290,), (0,), (23290,))

In [3]:
#Featurization
dataset = MorganFingerprint().featurize(dataset)
dataset.get_shape()

Featurizing datapoint 0
Featurizing datapoint 1000
Featurizing datapoint 2000
Featurizing datapoint 3000
Featurizing datapoint 4000
Featurizing datapoint 5000
Featurizing datapoint 6000


RDKit ERROR: [12:02:29] Explicit valence for atom # 1 Cl, 4, is greater than permitted


error in smile: O=[Cl]=O
Featurizing datapoint 7000


RDKit ERROR: [12:02:32] Explicit valence for atom # 3 B, 4, is greater than permitted
RDKit ERROR: [12:02:32] Explicit valence for atom # 1 Cl, 9, is greater than permitted


error in smile: OB1O[B]2(O)OB(O)O[B](O)(O1)O2
error in smile: O=[Cl-](=O)(=O)=O
Featurizing datapoint 8000
Featurizing datapoint 9000
Featurizing datapoint 10000
Featurizing datapoint 11000
Featurizing datapoint 12000
Featurizing datapoint 13000
Featurizing datapoint 14000


RDKit ERROR: [12:02:48] Explicit valence for atom # 0 P, 11, is greater than permitted


error in smile: [P](OCC=C(C)C)(OCC=C(C)C)(=O)(OP(OCC=C(C)C)(OCC=C(C)C)=O)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)CC=C(C)C
Featurizing datapoint 15000
Featurizing datapoint 16000
Featurizing datapoint 17000
Featurizing datapoint 18000
Featurizing datapoint 19000
Featurizing datapoint 20000
Featurizing datapoint 21000
Featurizing datapoint 22000
Featurizing datapoint 23000
Elements with indexes:  [6257, 7708, 7709, 14244]  were removed due to the presence of NAs!
The elements in question are:  ['O=[Cl]=O' 'OB1O[B]2(O)OB(O)O[B](O)(O1)O2' 'O=[Cl-](=O)(=O)=O'
 '[P](OCC=C(C)C)(OCC=C(C)C)(=O)(OP(OCC=C(C)C)(OCC=C(C)C)=O)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)CC=C(C)C']
(23286,) (23286,) (23286, 1024) (23286,)


((23286,), (23286,), (23286, 1024), (23286,))

In [4]:
#Feature Selection
dataset = LowVarianceFS(0.15).featureSelection(dataset)
dataset.get_shape()

(23286,) (23286,) (23286, 49) (23286,)


((23286,), (23286,), (23286, 49), (23286,))

In [5]:
#Data Split
splitter = SingletaskStratifiedSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset=dataset, frac_train=0.6, 
                                                                             frac_valid=0.2, frac_test=0.2)

In [6]:
#Scikit-Learn Random Forest
rf = RandomForestClassifier()
model = SklearnModel(model=rf)

In [7]:
#cross validation
model.cross_validate(dataset, Metric(roc_auc_score), folds=3)

Computing Stratified K-fold split
Train Score: 
roc_auc_score: 
 0.8502651026262165
Test Score: 
roc_auc_score: 
 0.642668731841772
Train Score: 
roc_auc_score: 
 0.85604581626905
Test Score: 
roc_auc_score: 
 0.6449339194768196
Train Score: 
roc_auc_score: 
 0.8597454064316816
Test Score: 
roc_auc_score: 
 0.6588125483768156


(SklearnModel(model=RandomForestClassifier(), model_dir='/tmp/tmpqjy7w3dw'),
 0.8597454064316816,
 0.6588125483768156,
 [0.8502651026262165, 0.85604581626905, 0.8597454064316816],
 [0.642668731841772, 0.6449339194768196, 0.6588125483768156],
 0.855352108442316,
 0.6488050665651358)

In [8]:
# model training
model.fit(train_dataset)

In [9]:
metrics = [Metric(roc_auc_score), Metric(precision_score), Metric(accuracy_score), Metric(confusion_matrix), 
           Metric(classification_report)]
print("#############################")
# evaluate the model
print('Training Dataset: ')
train_score = model.evaluate(train_dataset, metrics)
print("#############################")
print('Validation Dataset: ')
valid_score = model.evaluate(valid_dataset, metrics)
print("#############################")
print('Test Dataset: ')
test_score = model.evaluate(test_dataset, metrics)
print("#############################")

#############################
Training Dataset: 
roc_auc_score: 
 0.8630461758351012
precision_score: 
 0.9171974522292994
accuracy_score: 
 0.9810280641466208
confusion_matrix: 
 [[13127    52]
 [  213   576]]
classification_report: 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99     13179
           1       0.92      0.73      0.81       789

    accuracy                           0.98     13968
   macro avg       0.95      0.86      0.90     13968
weighted avg       0.98      0.98      0.98     13968

#############################
Validation Dataset: 
roc_auc_score: 
 0.6570905666550397
precision_score: 
 0.54375
accuracy_score: 
 0.946520618556701
confusion_matrix: 
 [[4320   73]
 [ 176   87]]
classification_report: 
               precision    recall  f1-score   support

           0       0.96      0.98      0.97      4393
           1       0.54      0.33      0.41       263

    accuracy                           0.95      46

In [10]:
#Build a model function for hyperparameter optimization
def rf_model_builder(n_estimators, max_features, class_weight, model_dir=None):
    rf_model = RandomForestClassifier(n_estimators=n_estimators, max_features=max_features, 
                                      class_weight=class_weight)
    return SklearnModel(rf_model, model_dir)

params_dict_rf = {"n_estimators": [10, 100],
                  "max_features": ["auto", "sqrt", "log2", None],
                  "class_weight": [{0: 1., 1: 1.}, {0: 1., 1: 5}, {0: 1., 1: 10}]
                  }
    


In [11]:
#Hyperparameter Optimization
optimizer = GridHyperparamOpt(rf_model_builder)

best_rf, best_hyperparams, all_results = optimizer.hyperparam_search(params_dict_rf, train_dataset, valid_dataset, Metric(roc_auc_score))

print('#################')
print(best_hyperparams)
print(best_rf)

Fitting 15 random models from a space of 24 possible models.
Fitting model 1/15
hyperparameters: {'n_estimators': 10, 'max_features': 'auto', 'class_weight': {0: 1.0, 1: 1.0}}
roc_auc_score: 
 0.6521163551761833
Model 1/15, Metric roc_auc_score, Validation set 1: 0.652116
	best_validation_score so far: 0.652116
Fitting model 2/15
hyperparameters: {'n_estimators': 10, 'max_features': 'auto', 'class_weight': {0: 1.0, 1: 5}}
roc_auc_score: 
 0.6836515749650109
Model 2/15, Metric roc_auc_score, Validation set 2: 0.683652
	best_validation_score so far: 0.683652
Fitting model 3/15
hyperparameters: {'n_estimators': 10, 'max_features': 'auto', 'class_weight': {0: 1.0, 1: 10}}
roc_auc_score: 
 0.7018788099629639
Model 3/15, Metric roc_auc_score, Validation set 3: 0.701879
	best_validation_score so far: 0.701879
Fitting model 4/15
hyperparameters: {'n_estimators': 10, 'max_features': 'sqrt', 'class_weight': {0: 1.0, 1: 5}}
roc_auc_score: 
 0.6801569901649617
Model 4/15, Metric roc_auc_score, Val

In [12]:
#Evaluate model
best_rf.evaluate(test_dataset, metrics)

roc_auc_score: 
 0.663789534691174
precision_score: 
 0.35661764705882354
accuracy_score: 
 0.9265463917525774
confusion_matrix: 
 [[4217  175]
 [ 167   97]]
classification_report: 
               precision    recall  f1-score   support

           0       0.96      0.96      0.96      4392
           1       0.36      0.37      0.36       264

    accuracy                           0.93      4656
   macro avg       0.66      0.66      0.66      4656
weighted avg       0.93      0.93      0.93      4656



{'roc_auc_score': 0.663789534691174,
 'precision_score': 0.35661764705882354,
 'accuracy_score': 0.9265463917525774,
 'confusion_matrix': 1164.0,
 'classification_report': None}